In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
from url_sequences.sequence_manager import *

In [2]:
filepath = "/home/chris/Scrivania/url2vec/cs.illinois.eduNoConstraint.words1000.depth.10/vertex.txt"
content_map = get_content_map(filepath)

In [ ]:
# to dowload english stopwords
nltk.download()

In [3]:
stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer("english")

In [4]:
tokens_map = to_tokens_map(content_map)
stems_map = to_stems_map(content_map,stemmer)

totalvocab_stemmed = get_total_vocab(stems_map)
totalvocab_tokenized = get_total_vocab(tokens_map)